In [ ]:
import pandas as pd

# Default History - Variables that depcit history of default (Red)
# acc_now_delinq - The number of accounts on which the borrower is now delinquent.

# Income - Variables related to income and cash flow (Orange)
# avg_cur_bal - Average current balance of all accounts
# open to buy - The difference between the credit limit assigned to a cardholder account and the present balance on the account.